In [51]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlayWrightTimeout
import time
import pandas as pd 

In [69]:
SEASONS = list(range(2017,2024)) #constant that captures seasons 2019-2020 to 2023-2024



In [71]:
DATA_DIR = "data0" # directory storing all the data

STANDINGS_DIR = os.path.join(DATA_DIR, "standings") # a directory that shows the standings that are scrapped as standings from the html

SCORES_DIR = os.path.join(DATA_DIR, "scores") # directory to store all the scores

In [73]:
async def get_html (url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1): # retries are performed by interval of 1 
        time.sleep(sleep* i) # time between retries is done by mutilple of 5 secs

        try:
            async with async_playwright() as p: 
                browser = await p.chromium.launch() # launches open source version of chrome
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlayWrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [75]:
async def scrape_season (season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    if html is None:
        print(f"Failed to retrieve HTML for season{season}")
        return
    
    soup = BeautifulSoup(html, "html.parser")
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
            save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
            if os.path.exists(save_path):
                continue
            
            html = await get_html(url,"#all_schedule")
            if html is None: 
                print(f"Failed to retrieve HTML for {url}")
                continue
              
            with open(save_path, "w+") as f: 
                f.write(html)

In [77]:
for season in SEASONS:
    await scrape_season(season)


2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Refere

In [79]:
standings_files = os.listdir(STANDINGS_DIR) 

In [81]:
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2017_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2023_games-december.html',
 'NBA_2017_games-april.html',
 'NBA_2022_games-november.html',
 'NBA_2018_games-october.html',
 'NBA_2017_games-march.html',
 'NBA_2021_games-february.ht

In [63]:
async def scrape_game(standings_file):
    standings_file = os.path.join(STANDINGS_DIR, standings_files[0])
    
    with open(standings_file, 'r') as f: 
        html = f.read()
    
    #Creating ariables for parsing data 
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l.get('href')for l in links]
    boxscores = [l for l in href if l and 'boxscore' in l and '.html' in l]
    box_scores = [f'https://www.basketball-reference.com{l}' for l in boxscores]
    
    #write path for boxscore 
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, '#content')
        if not html:
            continue
        with open(save_path,'w+') as f:
            f.write(html)

In [65]:
standings_file = [s for s in standings_files if '.html' in s] #remove any files that are not .html files

In [67]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Cavaliers vs Grizzlies, October 20, 2021 | Basketball-Reference.com
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
Magic vs Spurs, October 20, 2021 | Basketball-Reference.com
Thunder vs Jazz, October 20, 2021 | Basketball-Reference.com
Kings vs Trail Blazers, October 20, 2021 | Basketball-Reference.com
Nuggets vs Suns, October 20, 2021 | Basketball-Reference.com
Mavericks vs Hawks, October 21, 2021 | Basketball-Reference.com
Bucks vs Heat, October 21, 2021 | Basketball-Reference.com
Clippers vs Warriors, October 21, 2021 | Basketball-